In [ ]:
import subprocess as sub
path_to_grove = "/home/pk/projects/RAxMLGrove/"

selected_trees_file_path = "/home/pk/projects/RAxMLGrove/selected_grove_tree_ids_scalability"


selected_trees_file = open(selected_trees_file_path,"r")
# tree_ids = []
for line in selected_trees_file:    
    # tree_ids.append(line.split(",")[0])
    tree_id = line.split(",")[0]
    sub.call("cp -r " + path_to_grove + "trees/" + tree_id + " " + path_to_grove + "selected_trees/", shell=True)

# print(tree_ids)
selected_trees_file.close()



In [15]:
from ete3 import Tree
import numpy as np
from numpy.random import uniform, choice, dirichlet
import sys
import os

path_to_grove = "/home/pk/projects/RAxMLGrove/"
dna_trees_file_path = "/home/pk/projects/RAxMLGrove/all_gtr_dna_one_partition_trees"

num_trees_with_saturated_branches = 0
trees_binned = {}

branch_length_thresholds = [pow(10,-4),pow(10,-3),pow(10,-2),pow(10,-1),0.2,0.3,0.4,0.5,0.6,0.7,0.8]

for thresh in branch_length_thresholds:
    trees_binned[thresh] = []

dna_trees_file = open(dna_trees_file_path,"r")
trees_not_parsed = []
max_recorded_branch_length = 0
id_of_tree_with_largest_branch = 0
for line in dna_trees_file:
    tree_file_path = path_to_grove + "trees/" + line.strip() + "/tree_best.newick"
    size_bytes = os.path.getsize(tree_file_path)
    if (size_bytes > 0):
        try:
            t = Tree(tree_file_path, format=1)
            max_branch_length = max(node.dist for node in t.traverse() if not node.is_root())
            if max_recorded_branch_length < max_branch_length:
                id_of_tree_with_largest_branch = line.strip()
                max_recorded_branch_length = max_branch_length
            for thresh in branch_length_thresholds:
                if (max_branch_length < thresh):
                    trees_binned[thresh].append(line.strip())
                    break
        except:
            trees_not_parsed.append(line.strip())


In [8]:
print("number of trees not parsed is ", len(trees_not_parsed))

print(trees_not_parsed)

number of trees not parsed is  2
['36738', '5781']


In [9]:
trees_binned_file_path = "/home/pk/projects/RAxMLGrove/trees_binned_by_max_br_len"
trees_binned_file = open(trees_binned_file_path,"w")
for thresh in branch_length_thresholds:
    for tree_ind in trees_binned[thresh]:
        trees_binned_file.write(str(thresh)+"\t"+str(tree_ind)+"\n")
trees_binned_file.close()

In [10]:
import random

random.seed(101)

selected_trees = {}
for thresh in branch_length_thresholds:
    trees = trees_binned[thresh]
    if len(trees) <= 1000:
        # Take all if fewer than 1000
        selected_trees[thresh] = trees
    else:
        # Randomly select 1000 without replacement
        selected_trees[thresh] = random.sample(trees, 1000)



In [11]:
selected_trees_file_path = "/home/pk/projects/RAxMLGrove/selected_trees_binned_by_max_br_len"
selected_trees_binned_file = open(selected_trees_file_path,"w")
for thresh in branch_length_thresholds:
    for tree_ind in selected_trees[thresh]:
        selected_trees_binned_file.write(str(thresh)+"\t"+str(tree_ind)+"\n")
selected_trees_binned_file.close()

In [ ]:
# simulate sequences for selected trees
model_params = {"alpha":None,"Seq_Len":None,"ac":None,
         "ag":None,"at":None,"cg":None,"ct":None,"gt":None,
         "pi_a":None,"pi_c":None,"pi_g":None,"pi_t":None}
for tree_ind in selected_trees[thresh]:
    log_file_name = "/home/pk/projects/RAxMLGrove/trees/"+tree_ind+"/log_0.txt"
    log_file = open(log_file_name,"r")
    for line in log_file:
        if line.startswith("Substitution Matrix"):
            model = line.split(":")[1].strip()
            try:
                assert(model=="GTR")
                model_params["Substitution_Matrix"]                
            except:
                print(tree_ind)
                print(line)
                exit
            
    


In [14]:
print(id_of_tree_with_largest_branch)

max_recorded_branch_length

99998


0